## Wavefile Read/Write

Open and read a wavefile

In [1]:
from scipy.signal import kaiserord, firwin, freqz
from pylab import figure, plot, title, grid, xlabel, ylabel

class Filter:
    def __init__(self, filter_type='empty', nyquist_rate=22050, stop_band_attentuation_dB=10, low_transition_freq=400, high_transition_freq=4000):
        #! Adjust stop_band_attentuation_dB to control signal equalization
        self.filter_type = filter_type
        self.nyquist_rate = nyquist_rate
        self.transition_width = 200 / nyquist_rate #! Assign 5000 value to variable
        self.stop_band_attenuation_dB = stop_band_attentuation_dB
        self.low_transition_freq = low_transition_freq
        self.high_transition_freq = high_transition_freq
        
        self.filter_coefs = []
        
        self.calculate_filter_order()
        self.generate_filter_coefs()
    
    
    def calculate_filter_order(self):
        self.FIR_order, self.beta = kaiserord(self.stop_band_attenuation_dB, self.transition_width)
        #self.FIR_order *= 3
        if self.FIR_order % 2 == 0:
            self.FIR_order += 1
    
    
    def generate_filter_coefs(self):
        if self.filter_type == "low":
            self.generate_lowfreq_filter()
        elif self.filter_type == "mid":
            self.generate_midfreq_filter()
        elif self.filter_type == "high":
            self.generate_highfreq_filter()
        elif self.filter_type == 'empty':
            pass
    
    
    def generate_lowfreq_filter(self):
        self.filter_coefs = firwin(self.FIR_order, self.low_transition_freq, pass_zero='lowpass', fs=self.nyquist_rate*2)

    
    def generate_midfreq_filter(self):
        self.filter_coefs = firwin(self.FIR_order, [self.low_transition_freq / self.nyquist_rate, self.high_transition_freq / self.nyquist_rate], pass_zero='bandpass')

    
    def generate_highfreq_filter(self):
        self.filter_coefs = firwin(self.FIR_order, self.high_transition_freq, pass_zero='highpass', fs=self.nyquist_rate*2)

    
    def plot_filter(self):
        figure()
        w, h = freqz(self.filter_coefs, worN=8000)
        plot((w/np.pi)*self.nyquist_rate, np.absolute(h), linewidth=2)
        title('Filter Curves')
        xlabel("Frequency")
        ylabel("Filter Gain")
        grid(True)
        

print("Done")

Done


In [2]:
import wave
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter

from pynq import Overlay
from pynq import allocate

signal_input_file_path = './electric_guitar_sample.wav'
signal_output_file_path = './outputs/TEST_WEEE.wav'

class Audio_Signal:
    def __init__(self, input_file_path, output_file_path):
        self.input_file_path = input_file_path
        self.output_file_path = output_file_path

    
    def load_audio(self):
        audio_signal = wave.open(self.input_file_path, 'r')
        
        self.frame_count = audio_signal.getnframes()
        self.channel_count = audio_signal.getnchannels()
        self.sample_width = audio_signal.getsampwidth()
        self.sample_rate = audio_signal.getframerate()
        self.frame_rate = audio_signal.getframerate() * 2

        self.audio_frames = audio_signal.readframes(self.frame_count)
        self.audio_frames = np.frombuffer(self.audio_frames, dtype = np.int16)
    
    
    def plot_audio_data(self):
        audio_length = len(self.audio_frames)

        self.time_signal = np.linspace(0, audio_length / self.frame_rate, num = audio_length)

        plt.figure()
        plt.plot(self.time_signal, self.audio_frames)
        plt.xlabel("Time (s)")
        plt.ylabel("Signal Amplitude")

        plt.show()
        
        
    def SW_equalize(self, lowfreq_gain, midfreq_gain, highfreq_gain):
        self.make_coefs(lowfreq_gain, midfreq_gain, highfreq_gain)
        self.sw_equalized = lfilter(self.coefs.filter_coefs, 1.0, self.audio_frames)
        
        self.write_to_wav_file(self.sw_equalized)
        
        print(self.coefs.filter_coefs)
    
    
    def equalize(self, lowfreq_gain, midfreq_gain, highfreq_gain):
        overlay = Overlay('/home/xilinx/pynq/overlays/equalizer/equalizer.bit')
        #overlay?

        hls_ip = overlay.axi_dma_0

        CONTROL_REGISTER = 0x0
        hls_ip.write(CONTROL_REGISTER, 0x01)
        overlay.axi_dma_0.register_map
        
        dma = overlay.axi_dma_0
        self.input_buffer = allocate(shape=(4000,), dtype=np.int32)
        self.output_buffer = allocate(shape=(4000 - 35,), dtype=np.int32)
        
        self.make_coefs(lowfreq_gain, midfreq_gain, highfreq_gain)
        
        print(self.coefs.filter_coefs)

        self.format_input()

        print(0)
        dma.sendchannel.transfer(self.input_buffer)
        print(1)
        dma.recvchannel.transfer(self.output_buffer)
        print(2)
        dma.sendchannel.wait()
        print(3)
        dma.recvchannel.wait()
        print(4)
        
        self.equalized_frames = []
        
        for frame in range(4000 - 35):
            self.equalized_frames.append(self.output_buffer[frame] / 1000000)
            
        self.write_to_wav_file(self.equalized_frames)

    
    def make_coefs(self, lowfreq_gain, midfreq_gain, highfreq_gain):
        self.lowfreq_filter = Filter('low')
        self.midfreq_filter = Filter('mid')
        self.highfreq_filter = Filter('high')

        self.coefs = Filter()
        self.coefs.filter_coefs = (self.lowfreq_filter.filter_coefs * lowfreq_gain
                                   + self.midfreq_filter.filter_coefs * midfreq_gain
                                   + self.highfreq_filter.filter_coefs * highfreq_gain)
    
    
    def format_input(self):
        self.input_buffer[0] = 0xBEEF
        
        for coef in range(33):
            self.input_buffer[coef + 1] = coef
            self.input_buffer[coef + 1] = self.coefs.filter_coefs[coef] * 1000

        self.input_buffer[34] = 0xABBA

        for value in range(4000 - 35):
            self.input_buffer[value + 35] = self.audio_frames[value] * 1000

        print(self.input_buffer)
    
    
    def split_signal(self):
        pass
    
    
    def run_filter(self, dma):
        print(0)
        dma.sendchannel.transfer(self.input_buffer)
        print(1)
        dma.recvchannel.transfer(self.output_buffer)
        print(2)
        dma.sendchannel.wait()
        print(3)
        dma.recvchannel.wait()
        print(4)
        
    
    def write_to_wav_file(self, frames):
        self.output_frames = np.array(frames).astype(np.int16)
        
        with wave.open(self.output_file_path, 'w') as output_file:
            output_file.setnchannels(self.channel_count)
            output_file.setsampwidth(self.sample_width)
            output_file.setframerate(self.sample_rate)
            output_file.writeframes(self.output_frames)


print("Done")

Done


In [3]:
input_signal = Audio_Signal(signal_input_file_path, signal_output_file_path)

input_signal.load_audio()

input_signal.SW_equalize(0, 1, 0)

input_signal.equalize(0, 1, 0)

for i in range(100):
    print(input_signal.sw_equalized[i], " ", input_signal.equalized_frames[i])

[-8.22813863e-04  2.76067447e-05  7.63966314e-04  9.27854364e-04
 -6.28032204e-04 -5.22797084e-03 -1.33614463e-02 -2.36530699e-02
 -3.24460139e-02 -3.44540742e-02 -2.44775747e-02  3.65623679e-04
  3.88046844e-02  8.48632840e-02  1.29035762e-01  1.60944202e-01
  1.72581950e-01  1.60944202e-01  1.29035762e-01  8.48632840e-02
  3.88046844e-02  3.65623679e-04 -2.44775747e-02 -3.44540742e-02
 -3.24460139e-02 -2.36530699e-02 -1.33614463e-02 -5.22797084e-03
 -6.28032204e-04  9.27854364e-04  7.63966314e-04  2.76067447e-05
 -8.22813863e-04]


[-8.22813863e-04  2.76067447e-05  7.63966314e-04  9.27854364e-04
 -6.28032204e-04 -5.22797084e-03 -1.33614463e-02 -2.36530699e-02
 -3.24460139e-02 -3.44540742e-02 -2.44775747e-02  3.65623679e-04
  3.88046844e-02  8.48632840e-02  1.29035762e-01  1.60944202e-01
  1.72581950e-01  1.60944202e-01  1.29035762e-01  8.48632840e-02
  3.88046844e-02  3.65623679e-04 -2.44775747e-02 -3.44540742e-02
 -3.24460139e-02 -2.36530699e-02 -1.33614463e-02 -5.22797084e-03
 -6.28032204e-04  9.27854364e-04  7.63966314e-04  2.76067447e-05
 -8.22813863e-04]
[   48879        0        0 ... -3444000 -3751000 -3481000]
0
1
2
3
4
0.0   0.0
0.0   0.0
-0.03455818225010005   0.0
-0.03010744352211498   0.0
-0.03762635073460413   0.0
0.007018115796388877   0.0
-0.030257866597605085   0.0
-0.1966766486739983   -0.21
-0.7822384154976915   -0.736
-1.9126468911025407   -1.89
-3.8268374093820876   -3.721
-6.3669961781024496   -6.273
-9.293367903563022   -9.093
-11.832403633013891   -11.663
-13.188523636709107   -12.938
-12.2

In [ ]:
input_signal = Audio_Signal(signal_input_file_path, signal_output_file_path)

input_signal.load_audio()

In [ ]:
lowfreq_filter = Filter('low')
midfreq_filter = Filter('mid')
highfreq_filter = Filter('high')

# lowfreq_filter.plot_filter()
# midfreq_filter.plot_filter()
# highfreq_filter.plot_filter()

new_filter = Filter()
new_filter.filter_coefs = lowfreq_filter.filter_coefs * 0.6 + midfreq_filter.filter_coefs * 0.3 + highfreq_filter.filter_coefs * 0.4
new_filter.plot_filter()

## Hardware Function Testing

In [ ]:
from pynq import Overlay
import numpy as np
from pynq import allocate

overlay = Overlay('/home/xilinx/pynq/overlays/equalizer/equalizer.bit')
overlay?

## Hardware Function Testing with Sample Input

In [ ]:
dma = overlay.equalizerDMA.axi_dma_0
input_buffer = allocate(shape=(200,), dtype=np.int32)
output_buffer = allocate(shape=(99,), dtype=np.int32)

In [ ]:
input_buffer[0] = 0xBEEF

for i in range(99):
    input_buffer[i + 1] = i * ((-1) ** i)

input_buffer[100] = 0xABBA

for i in range(99):
    input_buffer[i + 101] = 1

print(input_buffer)

In [ ]:
def run_kernel():
    print(0)
    dma.sendchannel.transfer(input_buffer)
    print(1)
    dma.recvchannel.transfer(output_buffer)
    print(2)
    dma.sendchannel.wait()
    print(3)
    dma.recvchannel.wait()
    print(4)

In [ ]:
run_kernel()

In [ ]:
print(output_buffer)

In [37]:
from pynq import Overlay
import numpy as np
from pynq import allocate
import time

overlay = Overlay('/home/xilinx/pynq/overlays/equalizer/equalizer.bit')
#overlay?
hls_ip = overlay.axi_dma_0

CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x01)
overlay.axi_dma_0.register_map

dma = overlay.axi_dma_0
input_buffer = allocate(shape=(4000,), dtype=np.int32)
output_buffer = allocate(shape=(4000 - 35,), dtype=np.int32)

# input_buffer_2 = allocate(shape=(200,), dtype=np.int32)

input_buffer[0] = 0xBEEF

for i in range(33):
    input_buffer[i + 1] = i

input_buffer[34] = 0xABBA

for i in range(99):
    input_buffer[i + 35] = 1

print(input_buffer)
print(len(input_buffer))

# for i in range(200):
#     input_buffer_2[i] = 1
    
# print(input_buffer_2)

print(0)
dma.sendchannel.transfer(input_buffer)
print(1)
dma.recvchannel.transfer(output_buffer)
print(2)
dma.sendchannel.wait()
print(3)
dma.recvchannel.wait()
print(4)

print(output_buffer)

[48879     0     1 ...     0     0     0]
4000
0
1
2
3
4
[0 1 3 ... 0 0 0]
